In [1]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
from datetime import datetime
from pathlib import Path
import os
import time

In [2]:
key = 'AU74VSFGT1S37O4A'
stockSymbol = ["AAPL", "AMZN", "GOOGL","MSFT", "DELL", "IBM", "INTC", "HPQ",
               "FB", "CSCO", "ORCL", "HPE", "MU", "DXC", "TMO"]

In [3]:
#This gets the parent directory of the current directory
current = os.getcwd()
path = Path(current)
repo_folder = path.parent
print(repo_folder)

path_to_Values = str(repo_folder)+'\\data\\MinuteValues'
print(path_to_Values)

C:\Users\Russell\OneDrive - Marist College\School\Spring 2020\DATA 450 Data Capping\Repository
C:\Users\Russell\OneDrive - Marist College\School\Spring 2020\DATA 450 Data Capping\Repository\data\MinuteValues


In [4]:
for symbol in stockSymbol:
    
    newDf = pd.DataFrame(columns =["open", "high", "low", "close", 'volume','time'])
    
    
    url = ('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=1min&outputsize=full&apikey='+key).format(symbol)
    stockRes = requests.get(url).json()
    
    file = path_to_Values+'\\{}Values.csv'.format(symbol)
    df=pd.read_csv(file)
    df.time = pd.to_datetime(df.time)
    
    for times in stockRes['Time Series (1min)'].keys():
        timestamp = datetime.strptime(times, '%Y-%m-%d %H:%M:%S')
        ope = stockRes['Time Series (1min)'][times]['1. open']
        high = stockRes['Time Series (1min)'][times]['2. high']
        low = stockRes['Time Series (1min)'][times]['3. low']
        close = stockRes['Time Series (1min)'][times]['4. close']
        volume = stockRes['Time Series (1min)'][times]['5. volume']
        
        
        newDf = newDf.append({"open":ope, "high":high, "low":low, "close":close, "volume":volume, "time":timestamp}, ignore_index=True)
    
    dataframeValue = newDf.append(df)
    
    dataframeValue = dataframeValue[{'open', 'high', 'low', 'close', 'volume', 'time'}]
    
    dataframeValue['time'] = pd.to_datetime(dataframeValue['time'])
    
    dataframeValue = dataframeValue.drop_duplicates(subset='time')
    
    dataframeValue = dataframeValue.sort_values(by = 'time', ascending = False)
    
    dataframeValue.to_csv(file, index=False)
    time.sleep(60)

In [26]:
for symbol in stockSymbol:
    oldFile = path_to_oldValues+'\\{}Values.csv'.format(symbol)
    newFile = path_to_newValues+'\\{}Values.csv'.format(symbol)
    oldDf = pd.read_csv(oldFile)
    oldDf.time = pd.to_datetime(oldDf.time)
    newDf = oldDf.sort_values(by = 'time', ascending = False)
    newDf.to_csv(newFile, index=False)

In [28]:
dailyUrl = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={}&outputsize=full&apikey='+key.format(symbol)

In [17]:
for symbol in stockSymbol:
    dailyUrl = ('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={}&outputsize=full&apikey='+key).format(symbol)
    stockRes = requests.get(dailyUrl).json()
    
    file = path_to_newValues+'\\{}Daily.csv'.format(symbol)
    
    newDf = pd.DataFrame(columns =["open", "high", "low", "close", 'volume','time'])
    
    
    for times in stockRes['Time Series (Daily)'].keys():
        timestamp = datetime.strptime(times, '%Y-%m-%d')
        ope = stockRes['Time Series (Daily)'][times]['1. open']
        high = stockRes['Time Series (Daily)'][times]['2. high']
        low = stockRes['Time Series (Daily)'][times]['3. low']
        close = stockRes['Time Series (Daily)'][times]['4. close']
        volume = stockRes['Time Series (Daily)'][times]['5. volume']
        
        newDf = newDf.append({"open":ope, "high":high, "low":low, "close":close, "volume":volume, "time":timestamp}, ignore_index=True)
    
    dataframeValue = newDf
    
    dataframeValue = dataframeValue[{'open', 'high', 'low', 'close', 'volume', 'time'}]
    
    dataframeValue['time'] = pd.to_datetime(dataframeValue['time'])
    
    dataframeValue = dataframeValue.drop_duplicates(subset='time')
    
    dataframeValue.to_csv(file, index=False)
    time.sleep(60)

In [35]:
stockRes

{'Error Message': 'Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY.'}